In [1]:
%reload_ext autoreload
%autoreload 2

import metadrive
from metadrive.envs.base_env import BaseEnv
from metadrive.obs.observation_base import ObservationBase
from metadrive.constants import DEFAULT_AGENT
from metadrive.utils import Config
from metadrive.manager.scenario_map_manager import ScenarioMapManager
from metadrive.manager.base_manager import BaseManager
from metadrive.scenario.scenario_description import ScenarioDescription
from metadrive.manager.scenario_light_manager import ScenarioLightManager
from metadrive.manager.scenario_traffic_manager import ScenarioTrafficManager
from metadrive.component.vehicle_navigation_module.trajectory_navigation import TrajectoryNavigation



In [2]:
import scenario_to_metadrive
import waymo_loader

2023-09-01 18:38:58.129501: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-01 18:38:58.169776: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-01 18:38:58.170649: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-01 18:38:59.194155: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import os
import pickle

path = "~/data/waymo/training_20s.tfrecord-00000-of-01000"
path = os.path.expanduser(path)

print("Loading scenarios from Waymo dataset...")
scenarios = waymo_loader.parse_file(path)
print(f"Loaded {len(scenarios)} scenarios")

Loading scenarios from Waymo dataset...


2023-09-01 18:39:00.022910: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-01 18:39:00.044359: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/fidgetsinner/venvs/metadrive/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/fidgetsinner/venvs/metadrive/lib/python3.11/site-packages/numpy/core/_method

Loaded 61 scenarios


In [4]:
data = scenario_to_metadrive.convert_scenario(scenarios[0])

In [9]:
class InMemoryScenarioDataManager(BaseManager):
    def __init__(self, scenario):
        super().__init__()
        self.current_scenario = scenario
    
    def get_scenario(self, i, should_copy=False):
        return self.current_scenario

class ReplayEnv(BaseEnv):
    @classmethod
    def default_config(cls) -> Config:
        config = super().default_config()
        config.update({
            "start_seed": 0,
            "num_scenarios": 1,
            "start_scenario_index": 0,
            "no_map": False,
            "store_map": False,
            "need_lane_localization": True,
            "vehicle_config": dict(
                lidar=dict(num_lasers=120, distance=50),
                lane_line_detector=dict(num_lasers=0, distance=50),
                side_detector=dict(num_lasers=12, distance=50),
                show_dest_mark=True,
                navigation_module=TrajectoryNavigation,
            ),
            "max_lateral_dist": 4,
            # whether or not to base vehicle class purely on size or whether to evenly sample from all vehicle classes
            "even_sample_vehicle_class": False,
            # do show traffic lights
            "no_light": False,
            # if true, then any vehicle that is overlapping with another vehicle will be filtered 
            "filter_overlapping_car": False,
            # whether to use the default vehicle model
            "default_vehicle_in_traffic":False,
        })
        return config

    def __init__(self, config, scenario):
        super().__init__(config)
        self.scenario = scenario

    def done_function(self, vehicle_id: str):
        return False, {}

    def cost_function(self, vehicle_id: str):
        return 0, {}
    
    def reward_function(self, vehicle_id: str):
        return 0, {}

    def _get_observations(self):
        return {DEFAULT_AGENT: self.get_single_observation()}
    
    def setup_engine(self):
        self.engine.register_manager("agent_manager", self.agent_manager)
        self.engine.register_manager("map_manager", ScenarioMapManager())
        self.engine.register_manager("scenario_traffic_manager", ScenarioTrafficManager())
        self.engine.register_manager("scenario_light_manager", ScenarioLightManager()) 
        self.engine.register_manager("data_manager", InMemoryScenarioDataManager(self.scenario))


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1375608633.py, line 27)

In [6]:
env = ReplayEnv(config={"use_render": True, "manual_control": False}, scenario=ScenarioDescription(data))

[INFO] MetaDrive version: 0.4.1.2
[INFO] Sensors: [dashboard: VehiclePanel(), main_camera: MainCamera(1200, 900)]
[INFO] Render Mode: onscreen


In [7]:
env.reset()
for i in range(1000):
    env.step([0, 1])

[INFO] Assets version: 0.4.1.2


Known pipe types:
  glxGraphicsPipe
(1 aux display modules not yet loaded.)
:device(warning): /dev/input/event7 is not readable, some features will be unavailable.
:device(warning): /dev/input/event22 is not readable, some features will be unavailable.
:device(warning): /dev/input/event23 is not readable, some features will be unavailable.


KeyError: "'even_sample_vehicle_class' does not exist in existing config. Please use config.update(...) to update the config. Existing keys: dict_keys(['random_agent_model', 'agent_policy', 'num_agents', 'is_multi_agent', 'allow_respawn', 'delay_done', 'return_single_space', 'manual_control', 'controller', 'decision_repeat', 'discrete_action', 'discrete_steering_dim', 'discrete_throttle_dim', 'use_multi_discrete', 'action_check', 'rgb_clip', 'rgb_to_grayscale', 'stack_size', 'image_observation', 'use_render', 'debug', 'disable_model_compression', 'cull_scene', 'use_chase_camera_follow_lane', 'camera_height', 'camera_dist', 'camera_pitch', 'camera_smooth', 'camera_fov', 'prefer_track_agent', 'draw_map_resolution', 'top_down_camera_initial_x', 'top_down_camera_initial_y', 'top_down_camera_initial_z', 'show_logo', 'vehicle_config', 'sensors', 'auto_resize_window', 'target_vehicle_configs', 'window_size', 'physics_world_step_size', 'show_fps', 'global_light', 'debug_physics_world', 'debug_

In [8]:
env.close()